# <center> RESPIRATORY RATE ESTIMATION
 
 **08/02/2023**   
    
 **Group 7:** **Giovanni Zago**, **Enrico Lupi**, **Emanuele Sarte**, **Alessio Saccomani** 
 
   The aim of this project is to estimate the Respiratory Rate (RR) by Seismocardiography(SCG), a technique where the detector is positioned above sternum, and Ballistocardiography (BCG), in which there is no contact between the sensor and the body: in this study the sensor was placed on the subject chest directly on a sweater). The measurements were taken using the detector MuSe (Multi-sensor miniaturized, low-power, wireless Inertial Measurement Unit), provided by 221e (https://www.221e.com). An IMU is a combination of an accelerometer and a gyroscope sensor, capable of detecting movements and measuring their intensity in terms of acceleration and rotational speeds. Sometimes, like in this case, a magnetometer is also included. 

<table><tr>
<td> 
  <p align="center" style="padding: 10px">
    <img alt="Forwarding" src="https://www.researchgate.net/profile/Niccolo-Mora/publication/332138445/figure/fig2/AS:743069842407425@1554173064707/Positioning-of-IMU-sensor-and-ECG-electrodes-The-IMU-is-placed-over-the-subjects.ppm" width="320">
    <br>
    <em style="color: grey"></em>
  </p> 
</td>
<td> 
  <p align="center">
    <img alt="Routing" src="https://www.221e.com/wp-content/uploads/2022/10/221e-Muse1.jpg" width="515">
    <br>
    <em style="color: grey"></em>
  </p> 
</td>
</tr></table>


# Load Data and First Steps

First, we import the necessary libraries for the analysis (pandas, numpy, matplotlib, scipy, ecc.). 

We then load the datafile 'center_sternum.txt' as a pandas dataframe. For the following analysis we decide to take into account the linear acceleration (in mg), the angular velocity (in degrees per second) and the magnetic field (in mG) in all three directions; the quaternions are not considered, instead, so they are dropped. Afterwards, we calibrate the measurements using the information in the file README_1.txt and we add to the dataset the absolute time at which every measurement is taken: as the data collection frequency is 200 Hz, each measurement is taken every 5 ms. 

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy import fftpack
from scipy import fft
from scipy import signal
from scipy import optimize
from scipy import linalg
import seaborn as sns
import pywt
sns.set_theme(style="ticks")

In [2]:
filenames = ["center_sternum.txt", "1_Stave_supine_static.txt", "2_Mattress_supine.txt", 
             "3_Subject_sitting_chair.txt", "4_Chest_sweater.txt", "5_Under_chair.txt"]
data_cuts = [(6,70), (16, 80), (14, 112), (5, 65), (5, 145), (8, 90)]

FILE_CHOSEN = 4

T1_CUT = data_cuts[FILE_CHOSEN][0]
T2_CUT = data_cuts[FILE_CHOSEN][1]
file_name = filenames[FILE_CHOSEN]

In [3]:
raw_data = pd.read_csv(file_name, sep="\t")

# We drop the columns we are not going to use
raw_data = raw_data.drop(["Log Mode", "qw", "qi", "qj", "qk"], axis=1)

# We look that the data collection frequency is the same for all the dataset
if len(raw_data["Log Freq"].unique()) != 1:
    print("There is more than one frequency")
    exit(1)

# We select the frequency
ACQ_FREQ = raw_data.loc[0, "Log Freq"]

# We create a column with the absolute time by multiplying 1/freq by integers
raw_data.insert(2, "Abs Time", np.arange(0, len(raw_data)) * (1 / ACQ_FREQ), allow_duplicates=False)

display(raw_data)


,Log Freq,Timestamp,Abs Time,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,MagnX,MagnY,MagnZ
0,100,1555414552,0.00,687.836,380.823,580.354,-7.926829,8.048780,0.975610,-592,122,-113
1,100,1555414552,0.01,-812.703,-108.946,726.876,-11.707320,-81.829270,8.780488,125,60,-128
2,100,1555414552,0.02,-812.520,-108.824,733.952,-37.073170,-145.487800,14.756100,123,59,-136
3,100,1555414552,0.03,-787.144,-45.628,616.954,-56.341460,-174.634200,23.048780,118,58,-145
4,100,1555414552,0.04,-800.259,-137.433,411.689,-1.097561,37.073170,1.341463,113,57,-157
...,...,...,...,...,...,...,...,...,...,...,...,...
14933,100,1555415471,149.33,-890.722,-81.008,446.764,-2.682927,2.926829,0.365854,144,47,-50
14934,100,1555415471,149.34,-885.964,-76.006,459.025,-3.902439,3.292683,0.365854,146,45,-51
14935,100,1555415471,149.35,-879.681,-77.714,475.068,-3.048780,2.804878,0.365854,144,46,-54
14936,100,1555415471,149.36,-894.992,-76.311,502.701,0.121951,1.463415,0.609756,144,45,-53


In [4]:
#calibrate the dataset
offset_gyro = np.array([-2.242224, 2.963463, -0.718397])

calibration_acc = np.array([[1.000966,     -0.002326418,-0.0006995499],
                            [-0.002326379,  0.9787045,  -0.001540918],
                            [-0.0006995811,-0.001540928, 1.00403]])
offset_acc = np.array([-3.929942, -13.74679, 60.67546])

calibration_magn = np.array([[0.9192851, -0.02325168, 0.003480837],
                             [-0.02325175, 0.914876, 0.004257396],
                             [0.003481006, 0.004257583, 0.8748001]])
offset_magn = np.array([-95.67974, -244.9142, 17.71132])

if file_name != "center_sternum.txt":
    offset_gyro = np.array([-2.804399, 1.793105, 0.3411708])

    calibration_acc = np.array([[1.002982,    9.415505E-05, 0.004346743],
                                [9.04459E-05, 1.002731,    -0.001444198],
                                [0.004346536,-0.001444751,  1.030587]])
    offset_acc = np.array([3.602701, -20.96658, 54.97186])

    calibration_magn = np.array([[1.013437,    -0.04728858,  -0.001861475],
                                 [-0.04728862,  1.004832,    0.008222118],
                                 [-0.001861605, 0.008221965, 0.9439077]])
    offset_magn = np.array([-150.4098, 74.62431, 630.9805])

raw_data[['AccX', 'AccY', 'AccZ']] = np.dot(raw_data[['AccX', 'AccY', 'AccZ']], calibration_acc.T) + offset_acc.T
raw_data[['GyroX', 'GyroY', 'GyroZ']] = raw_data[['GyroX', 'GyroY', 'GyroZ']] + offset_gyro.T
raw_data[['MagnX', 'MagnY', 'MagnZ']] = np.dot(raw_data[['MagnX', 'MagnY', 'MagnZ']], calibration_magn.T) + offset_magn.T

display(raw_data)

,Log Freq,Timestamp,Abs Time,AccX,AccY,AccZ,GyroX,GyroY,GyroZ,MagnX,MagnY,MagnZ
0,100,1555414552,0.00,696.048334,360.120513,655.516657,-10.731228,9.841885,1.316781,-755.923364,224.279578,526.424080
1,100,1555414552,0.01,-808.374494,-131.333370,800.705773,-14.511719,-80.036165,9.121659,-26.329221,127.950721,510.420932
2,100,1555414552,0.02,-808.160179,-131.221239,807.998826,-39.877569,-143.694695,15.097271,-28.293915,126.974690,502.865171
3,100,1555414552,0.03,-783.211118,-67.681388,687.441203,-59.145859,-172.841095,23.389951,-33.297058,126.132302,494.371088
4,100,1555414552,0.04,-797.266105,-159.441850,475.973393,-3.901960,38.866275,1.682634,-38.294616,125.265247,483.045282
...,...,...,...,...,...,...,...,...,...,...,...,...
14933,100,1555415471,149.33,-887.841091,-102.921591,511.646512,-5.487326,4.719934,0.707024,-6.604362,114.630747,583.903476
14934,100,1555415471,149.34,-883.015136,-97.923207,524.295993,-6.706838,5.085788,0.707024,-4.481049,112.518283,582.939401
14935,100,1555415471,149.35,-876.643826,-99.658473,540.859477,-5.853179,4.597983,0.707024,-6.549627,113.593026,580.119623
14936,100,1555415471,149.36,-891.880238,-98.292933,569.269111,-2.682448,3.256520,0.950927,-6.504200,112.596416,581.055309


# Data Visualization

We now plot the accelerations, the angular velocity and the magnetic field components as a function of time in order to look at their general trend and to select a proper time window to conduct the analysis in. This is necessary as signals are very disturbed at the beginning and end of the data taking, probably due to sudden movements of the test subject: they are ususally used to coordinate the data taking across various devices, but they mud the data and need of course to be eliminated. 

From 6 to 70 seconds all features have a regular behaviour and thus we select this to be the time window to be used in the rest of the project. In the plots this time window is highlighted.

In [ ]:
from matplotlib.ticker import AutoMinorLocator, MultipleLocator, FuncFormatter

labels = ["GyroX", "GyroY", "GyroZ", "AccX", "AccY", "AccZ", "MagnX", "MagnY", "MagnZ"]
unit = ["[deg/s]", "[mg]", "[mG]"]
Nvar = len(labels)
   
IT1_CUT = round(T1_CUT * ACQ_FREQ)
IT2_CUT = round(T2_CUT * ACQ_FREQ) + 1

plt.figure(figsize=(18, 4 * (Nvar // 3)))
for i in range(Nvar // 3):
    ax = plt.subplot(Nvar // 3, 1, i + 1)
    plt.plot(raw_data["Abs Time"], raw_data[labels[i * 3]],     label=labels[i * 3])
    plt.plot(raw_data["Abs Time"], raw_data[labels[i * 3 + 1]], label=labels[i * 3 + 1])
    plt.plot(raw_data["Abs Time"], raw_data[labels[i * 3 + 2]], label=labels[i * 3 + 2])
    ymin, ymax = plt.ylim()
    plt.vlines(x=raw_data.loc[IT1_CUT, "Abs Time"], ymin=ymin, ymax=ymax, colors="black")
    plt.vlines(x=raw_data.loc[IT2_CUT, "Abs Time"], ymin=ymin, ymax=ymax, colors="black")

    ax.fill_between(raw_data.loc[IT1_CUT: IT2_CUT, "Abs Time"], ymin, ymax, color='C0', alpha=0.25)

    ax.xaxis.set_major_locator(MultipleLocator(10))
    ax.xaxis.set_minor_locator(AutoMinorLocator(5))
    plt.xlabel("Time [s]")
    plt.ylabel("Magnitude "+unit[i])
    plt.legend()

plt.show()

raw_data_cut = raw_data.loc[IT1_CUT: IT2_CUT - 1, :]
display(raw_data_cut)

# Statistical Analysis

We calculate the mean, standard deviation, minimum, 25th, 50th, 75th percentiles and maximum for all features (in the selected time window); data is then normalized. 
For the accelerations and the angular velocities the 25th and 75th percentiles are nearly symmetric, so the variables change quite regularly. 

Afterwards, normalized data is plotted in a 9x9 grid in order to highlight possible correlations: however, no significant correlation has been noticed. 

In [ ]:
raw_data_cut.drop(["Log Freq", "Timestamp", "Abs Time"], axis=1).describe()

In [ ]:
data_std = (raw_data_cut[labels] - np.mean(raw_data_cut[labels], axis=0)) / np.std(raw_data_cut[labels], axis=0)
data_std.insert(0, "Abs Time", raw_data_cut["Abs Time"])
data_std.set_index(np.arange(0, len(data_std)), inplace=True)
display(data_std)

In [ ]:
sns.pairplot(data_std[labels]);

# PCA

In order to reduce the dimensionality of the dataset we perform a **PCA**, keeping only 6 out of the total 9 principal components so as to maintain at least 85% of the total variability.

In [ ]:
avls, avts = linalg.eig(np.cov(data_std[labels].T))
sort_perm = np.flip(np.argsort(avls))

avls = np.real_if_close(avls[sort_perm])
avts = avts[:, sort_perm]

var_ratios = avls / np.sum(avls)
print('Eigenvalues (sorted):\n', np.round(avls, 4)) 
print('Variability ratios:\n', np.round(var_ratios * 100, 2))
print("Eigenvector:")
display(pd.DataFrame(avts, columns=[f"Avt {i+1}" for i in range(len(avls))]))

In [ ]:
PCA_LABELS = [f"PC{i+1}" for i in range(len(avls))]
data_rot = pd.DataFrame(data=np.dot(avts.T, data_std[labels].T).T, columns=PCA_LABELS)

perc_thr = .85
N_PCA = np.argmax(np.cumsum(var_ratios) >= perc_thr) + 1

print(f'To keep {int(perc_thr * 100)}% of variability in our dataset we need {N_PCA} out of the {len(var_ratios)} principal components')

display(data_rot)

PCA_LABELS = PCA_LABELS[:N_PCA]
data_pca = data_rot[data_rot.columns[:N_PCA]]
data_pca.insert(0, "Abs Time", data_std["Abs Time"])
display(data_pca)



# Fourier Analysis

We then perform a **Fourier analysis** using the scipy fft library in order to extrapolate the principal frequencies of the data: we expect to see a peak in the range [0.1, 0.3] Hz, the average respiratory rate in an adult.

We first plot as an example the whole power spectrum of PC3: we can clearly see a **peak around 0.2 Hz** due to respiration and one around 1.1 Hz corresponding to the heart rate, while the peaks at high frequencies are due to noise. As we are only interested in the respiratory rate, all following plots will be focused only in the [0, 0.9] Hz range.

In [ ]:
sig_fft = fft.fftshift(fft.fft(data_pca[PCA_LABELS], axis=0))
sample_freq = fft.fftshift(fft.fftfreq(len(data_pca), d=1/ACQ_FREQ))

In [ ]:
plt.figure(figsize=(18, 5))
ax = plt.subplot(1, 1, 1)
ax.plot(sample_freq, np.abs(sig_fft[:, 2]), label="Fourier Transform of " + PCA_LABELS[2])
ax.set_xlim(0, 10)
ax.set_xticks(np.arange(0, 10, 0.5))
plt.xlabel("Freq [Hz]")
plt.ylabel("Power")
plt.legend(loc="best");

In [ ]:
plt.figure(figsize=(18, 5 * N_PCA))
for i in range(N_PCA):
    ax = plt.subplot(N_PCA, 1, i+1)
    plt.plot(sample_freq, np.abs(sig_fft[:, i]), label="Frequency Spectrum " + PCA_LABELS[i])
    ax.set_xlim(0, 0.9)
    plt.xticks(np.arange(0, 1, 0.05))
        
    plt.xlabel("Freq [Hz]")
    plt.ylabel("Power")
    plt.legend(loc="best")
    

In [ ]:
sig_fft_w = np.zeros_like(sig_fft)
for i in range(N_PCA):
    sig_fft_w[:, i] = sig_fft[:, i]*var_ratios[i] 
total_FFT = np.sum(np.abs(sig_fft_w), axis=1)
plt.figure(figsize=(18, 5))
plt.plot(sample_freq, total_FFT, label="Frequency Spectrum of Weighted Sum of PCs")
plt.xlabel("Frequency [Hz]")
plt.ylabel("Power")
plt.xticks(np.arange(0, 0.9, 0.05))
plt.xlim(left=0, right=0.9)
plt.legend();

# Filter

We now apply a filter using a **wavelet transform** in order to extrapolate only the components of the signal at the desired frequencies.

Wavelet transform is a powerful alternative to Fourier transform, able not only to the frequency information of the dataset but also at which moment a particular frequency is present. In this project we used a dyadic tree-level wavelet decomposition via the wavelet **sym5**, as it is smooth and allows for a high level of decomposition. In the first level, the signal is split into low- and high-frequency components, that respectively provide the coarser part and the detailed information of the original signal and are thus termed as the approximation and the detail; this first low-frequency subband component is then downsampled by a factor of 2 and again decomposed into low- and high-frequency subbands. This process can be repeated to J levels as desired: in this project we stopped at level 10. The frequency range covered by AJ is $0$ to $f_{s}/2^{J+1}$ Hz and that for Dj is $f_{s}/2^{j+1}$ to $f_{s}/2^{j}$ Hz ($1\le j \le J$), where $f_{s}$ represents the sampling frequency (in unit Hz) of the original signal, so 200 Hz in our case. [Source](https://doi.org/10.1016/j.bspc.2019.101779) 

For this project we want to filter the high frquencies components due to heart beats and noise, and also very low, near-zero frequencies: we will thus reconstruct the time signal using only the coefficients for **D10** and **D9** which correspond to a range of [0.098, 0.39] Hz.

In [ ]:
#Filter signal using wavelets

lvl = 10
if file_name != "center_sternum.txt":
    lvl = 9

coeffs = pywt.wavedec(data_pca[PCA_LABELS], "sym5", level=lvl, axis=0)
#set coefficients related to undesired freuqncies to zero 
coeffs[0] = np.zeros_like(coeffs[0]) #set A10 to zero
for i in range(3, lvl + 1):
    coeffs[i] = np.zeros_like(coeffs[i]) #set Dj to zero with j from 8 to 1

#reconstruct time signal 
filtered_sig = pywt.waverec(coeffs, "sym5", axis=0)

In [ ]:
#plot recostructed time signal
fig, axs = plt.subplots(nrows=N_PCA, ncols=1, figsize=(18, N_PCA*5))

for i in range(N_PCA):
    axs[i].plot(data_pca["Abs Time"], filtered_sig[:-1, i], label=("Wavelet Filtered "+PCA_LABELS[i]))
        
    axs[i].set_xlabel("Time [s]")
    axs[i].set_ylabel("Magnitude") 
    axs[i].legend(loc="best")
    axs[i].xaxis.set_major_locator(plt.MultipleLocator(5))
    

We will now quickly repeat a Fourier analysis in order to show that indeed only the desired frequencies are kept after the filter.

In [ ]:
sig_fft_wt = fft.fftshift(fft.fft(filtered_sig[:-1, :], axis=0))
sample_freq = fft.fftshift(fft.fftfreq(len(data_pca), d=1/ACQ_FREQ))

plt.figure(figsize=(18, 5 * N_PCA))
for i in range(N_PCA):
    ax = plt.subplot(N_PCA, 1, i+1)
    plt.plot(sample_freq, np.abs(sig_fft[:, i]), label=PCA_LABELS[i])
    plt.plot(sample_freq, np.abs(sig_fft_wt[:, i]),
             label="Wavelet Filtered " + PCA_LABELS[i])
    ax.set_xlim(0, 0.9)
    plt.xticks(np.arange(0, 1, 0.05))
        
    plt.xlabel("Freq [Hz]")
    plt.ylabel("Power")
    plt.legend(loc="best")
    

# Metrics

### Peak Detection

After having filtered our signals we are ready to extract the **Respiratory Rate (RR)** and the **Respiratory Rate Variability (RRV)** from them. The idea is to spot the peaks of our signals, in order to compute the difference of their corresponding consecutive x-coordinate: this allows us to collect a sample of periods $T_i$ that can be further elaborated to estimate RR and RRV. To do so we start by naively using the find_peaks function of Scipy on each of our filtered PC signals. However, find_peaks, if used without appropriate parameters, returns raw peaks that do not reflect properly the periodicity of the signals, as it will be evident from the example plot below.

In [ ]:
filtered_sig=filtered_sig.T

In [ ]:
peaks = []
valleys = []
for i in range(len(filtered_sig)):
    temp_signal = filtered_sig[i][:-1]
    temp_peaks, _ = signal.find_peaks(temp_signal)
    temp_valleys, _ = signal.find_peaks(-temp_signal)
    peaks.append(temp_signal[temp_peaks])
    peaks.append(data_std['Abs Time'].values[temp_peaks])
    valleys.append(temp_signal[temp_valleys])
    valleys.append(data_std['Abs Time'].values[temp_valleys])
    
fig, ax = plt.subplots(1, 1, figsize=(18, 5 ))
    
ax.plot(data_std['Abs Time'], filtered_sig[1][:-1], label="Wavelet Filtered "+PCA_LABELS[1])
ax.plot(peaks[1 * 2 + 1], peaks[1 * 2], 'x', label="Peaks", markersize=10, markeredgewidth=3)
ax.set_xlabel("Time [s]")
ax.set_ylabel("Magnitude")
ax.legend(loc="best")
ax.xaxis.set_major_locator(plt.MultipleLocator(5))

'''
fig, ax = plt.subplots(len(filtered_sig), 1, figsize=(18, 5 * N_PCA))
for i in range(len(peaks) // 2):    
    ax[i].plot(data_std['Abs Time'], filtered_sig[i][:-1], label="Wavelet Filtered "+PCA_LABELS[i])
    ax[i].plot(peaks[i * 2 + 1], peaks[i * 2], 'x', label="Peaks", markersize=10, markeredgewidth=3)
    ax[i].set_xlabel("Time [s]")
    ax[i].set_ylabel("Magnitude")
    ax[i].legend(loc="best")
''';

### Improving Peak Detection

In order to improve the detection of the peaks we run the code again, paying attention to properly setting the parameters *height* and *distance* of the function find peaks. The former sets an height level that each sample must reach in order to be potentially recognized as a peak, and it has been set equal to the 70% of the mean y-difference between maxima and the following minima. The latter sets a minimum sample distance that needs to occur between subsequent peaks, and has been set equal to the 70% of the mean peak distance. In both cases we have set the 70% of the two quantities so as to preserve the variability of the $T_i$, otherwise small $T_i$ values would have been excessively penalized with respect to larger ones.

In [ ]:
avg_peak_dist = []
for i in range(1, len(peaks), 2):    
    avg_peak_dist.append(len(data_std['Abs Time']) / len(peaks[i]))

sign_amp = []
for i in range(1, len(peaks), 2):
    temp_sign_amp = []    
    for j in range(np.min([len(peaks[i]), len(valleys[i])])):
        temp_peaks = np.array(peaks[i-1])
        temp_valleys = np.array(valleys[i-1])
        temp_sign_amp.append(0.5*np.abs(temp_peaks[j] + temp_valleys[j]))
    sign_amp.append(temp_sign_amp)

avg_sign_amp = np.empty(shape=(len(filtered_sig)))
for i,x  in enumerate(sign_amp):
    avg_sign_amp[i] = np.mean(x)

height_perc = .6
distance_perc = .7
peaks_refined = []
valleys_refined = []
for i in range(len(filtered_sig)):
    temp_signal = filtered_sig[i][:-1]
    temp_peaks, _ = signal.find_peaks(temp_signal, height = height_perc * avg_sign_amp[i], distance = distance_perc * avg_peak_dist[i])
    temp_valleys, _ = signal.find_peaks(-temp_signal)
    peaks_refined.append(temp_signal[temp_peaks])
    peaks_refined.append(data_std['Abs Time'].values[temp_peaks])
    valleys_refined.append(temp_signal[temp_valleys])
    valleys_refined.append(data_std['Abs Time'].values[temp_valleys])

fig, ax = plt.subplots(len(filtered_sig), 1, figsize=(18, 5 * N_PCA))
for i in range(len(peaks_refined) // 2):    
    ax[i].plot(data_std['Abs Time'], filtered_sig[i][:-1], label="Wavelet Filtered "+PCA_LABELS[i])
    ax[i].plot(peaks_refined[i * 2 + 1], peaks_refined[i * 2], 'x', label="Peaks", markersize=10, markeredgewidth=3)
    ax[i].axhline(avg_sign_amp[i] * height_perc, color='g', label="Peak Height Threshold")
    ax[i].legend(loc="best")
    ax[i].xaxis.set_major_locator(plt.MultipleLocator(5))
    ax[i].set_xlabel("Time [s]")
    ax[i].set_ylabel("Magnitude")
    ax[i].legend(loc="lower right")

### Generate Histogram and Estimate Respiratory Rate

After detecting the peaks, the sample of $T_i$ is generated, plotted in an histogram and fitted with a gaussian function in order to estimate the Respiration Rate. 

In [ ]:
time_dist = []
for i in range(1, len(peaks_refined), 2):
    temp_time_dist = []    
    for j in range(1, len(peaks_refined[i])):
        temp_peaks_time = peaks_refined[i]
        temp_time_dist.append(temp_peaks_time[j]-temp_peaks_time[j-1])
    time_dist.append(temp_time_dist)

weights = [[var_ratios[i] for j in range(len(time_dist[i]))] for i in range(N_PCA)]

time_dist_total = np.concatenate(time_dist)
weights_total = np.concatenate(weights)
print(f'The set of the T_i contains {len(time_dist_total)} samples.')

fig, ax = plt.subplots(figsize=(8,6))
bins = ax.hist(x=time_dist_total, bins=round(np.sqrt(len(time_dist_total))),label="Respiratory Period",
               weights=weights_total, facecolor="lightblue", edgecolor="black")

bin_centers = (bins[1][:-1] + bins[1][1:]) / 2
bin_counts = bins[0]
bin_width = round(bins[1][1]-bins[1][0], 2)

ax.set_xticks(bin_centers)
ax.set_xticklabels(map(lambda x: round(x,1),bin_centers))

def my_gaus(x, A, mu, sigma):
    return A * np.exp(-0.5 * ((x-mu)/sigma) ** 2)

params, params_cov = optimize.curve_fit(my_gaus, bin_centers, bin_counts, p0=[1, np.mean(time_dist_total), np.std(time_dist_total)], absolute_sigma=True, bounds=(0,[100, 100, 100]))
fit_domain = np.sort(np.random.uniform(np.min(bin_centers), np.max(bin_centers), 1000))
ax.plot(fit_domain, my_gaus(fit_domain, *params), label="Gaussian Fit")
ax.set_xlabel("Respiratory Period [s]")
ax.set_ylabel(f"Counts / {bin_width} s")
errors = np.sqrt(bins[0])
ax.errorbar(x=bin_centers, y=bins[0], yerr=errors/2, fmt='.', capsize=1, 
            color='dimgrey', elinewidth=2, label="1/2 $\cdot$ Poisson Error")

ax.text(9.0, 2.9, f'$\mu$ = ({params[1]:.1f} $\pm$ {np.sqrt(np.diag(params_cov)[1]):.1f}) s \n$\sigma$ = ({params[2]:.1f} $\pm$ {np.sqrt(np.diag(params_cov)[2]):.1f}) s', fontsize = 14)

ax.legend()


print('Fit parameters (Amplitude, Mean, Sigma):', params)
print('Fit parameters errors (Amplitude, Mean, Sigma):', np.sqrt(np.diag(params_cov)))
print(f'The estimated Respiratory Period is {params[1]:.2f} s, corresponding to a Respiratory Rate of {1 / params[1]:.2f} Hz, i.e. approximately {60 / params[1]:.1f} breaths per minute.')

For the Respiratory Period estimate we choose the mean parameter returned by the gaussian fit, and we associate to it the respective error.

### Respiratory Rate Variability

Now that we got an estimate of the Respiratory Rate, we need an appropriate quantity that measures its variability. As we have found [here](https://www.ncbi.nlm.nih.gov/pmc/articles/PMC8339683/), there are many quantities that are used to estimate Respiratory Rate Variability, according to the specific setup one is working on. A common RRV parameter used in the case of quantitative time series analysis is root mean square of successive differences (RMSSD), defined as 
$$
\text{RMSSD}(\vec{x}) = \sqrt{\frac{1}{N}\sum_{i=2}^{N}|x_i - x_{i-1}|^2}
$$
where the vector $\vec{x}$ represents the set of $T_i$. However, to include in this calculation the information coming from the previous PCA, we decided to perform the weighted mean of the RMSSD of each selected PC, with the weights taken as the variability ratios of each PC. The equation becomes
$$
\text{RMSSD}(\vec{x}) = \frac{1}{\sum_{\text{selected PCs}} w_{PC}} \sum_{\text{selected PCs}} \sqrt{\frac{1}{N_{PC}}\sum_{i=2}^{N_{PC}} |x_i - x_{i-1}|^2} \cdot w_{PC}
$$
Contrary to the previous case, fitting the histogram of the set of successive differences with a gaussian was inconclusive.

In [ ]:
RRVs_sq = np.zeros(shape=len(time_dist))
for i, x in enumerate(time_dist):
    x = np.array(x)
    RRVs_sq[i] = np.sqrt(np.mean(np.diff(x) ** 2))

RRV = np.average(RRVs_sq, weights=var_ratios[:N_PCA])
print(f'The Respiratory Period Variability is {RRV:.2f} s.')

RRVs_set = []
for i, x in enumerate(time_dist):
    x = np.array(x)
    RRVs_set.append(np.abs(np.diff(x)))

weights = [[var_ratios[i] for j in range(len(RRVs_set[i]))] for i in range(N_PCA)]
weights_total = np.concatenate(weights)
RRVs_set = np.concatenate(RRVs_set)


fig, ax = plt.subplots(figsize=(8,6))
bins = ax.hist(x=RRVs_set, bins=round(np.sqrt(len(RRVs_set))),label="Respiratory Period",
               weights=weights_total, facecolor="lightblue", edgecolor="black")

bin_centers = (bins[1][:-1] + bins[1][1:]) / 2
bin_counts = bins[0]
bin_width = round(bins[1][1]-bins[1][0], 2)

ax.set_xticks(bin_centers)
ax.set_xticklabels(map(lambda x: round(x,1),bin_centers))

fit_domain = np.sort(np.random.uniform(np.min(bin_centers), np.max(bin_centers), 1000))
ax.set_xlabel("Respiratory Period Variability [s]")
ax.set_ylabel(f"Counts / {bin_width} s")
errors = np.sqrt(bins[0])
ax.errorbar(x=bin_centers, y=bins[0], yerr=errors/2, fmt='.', capsize=1, 
            color='dimgrey', elinewidth=2, label="1/2 $\cdot$ Poisson Error");
ax.legend();

## Extras

### Alternative Method without Filter

When the filter fails we can devise an alternative method. We first consider only the frequencies range [0.1, 0.25] Hz and select the 3 frequencies with maximum power within it; we the compute their mean and then we get a final estimate with the weighted average of these values between all the considered Principal Components.

In [ ]:
mask = (sample_freq > 0.1) & (sample_freq < 0.25)
power_fft = np.abs(sig_fft)
powers = power_fft[mask]
frequencies = sample_freq[mask]

list_freq=[]
power_list=[]

for j in range(N_PCA):
    for i in range(3):
        l=np.argmax(powers[:,j], axis=0)
        list_freq.append(frequencies[l])
        power_list.append(powers[l,j])
        powers[l,j]=0
        
freq_ = np.array(list_freq)
pow_ = np.array(power_list)
pow_ = pow_/(pow_.sum())
freq_ = freq_*pow_


RR = np.array(freq_).sum()

print(f"Respiratory Rate Per Minute is: {RR * 60:.2f}")

### Time Signal

Let us plot the features as a function of time individually, zooming in a restricted time window to highlight their trend. 

In [ ]:
fig, axs = plt.subplots(Nvar, 2, figsize=(18, 4 * Nvar))
for i in range(2):
    for j in range(Nvar):
        if i == 0:
            x_range = raw_data["Abs Time"]
            data_range = raw_data[labels[j]]
        else:
            xmax, xmin = 16, 24
            ixmin, ixmax = (xmax * ACQ_FREQ - 50,  xmin * ACQ_FREQ + 50)
            data_range = raw_data.loc[ixmin:ixmax ,labels[j]]
            x_range = raw_data.loc[ixmin:ixmax, "Abs Time"]
            axs[j][i].set_xlim(xmax, xmin)

        axs[j][i].plot(x_range, data_range, label=labels[j]) #c='b' ?
        axs[j][i].set_xlabel("Time [s]")

        if "Acc" in labels[j]:
            axs[j][i].set_ylabel("Linear Acceleration [mg]")
        elif "Gyro" in labels[j]:
            axs[j][i].set_ylabel("Angular Velocity [deg/s]")
        elif "Magn" in labels[j]:
            axs[j][i].set_ylabel("Magnetic Field [mG]")

        axs[j][i].legend(loc="best")

### Different Types of Filter

We tried different types of filters to find out which one worked best for our purposes.

First, we implemente a simple Butterworth filter in the range [0.1, 0.4] Hz, which has the merit of not having any ripple in the passband (at the expense of rolling slowly towards zero in the stopband).

In [ ]:
FREQ_RANGE = (0.1, 0.4)

sos = signal.butter(4, FREQ_RANGE, 'bandpass', fs=ACQ_FREQ, output='sos')
filtered_butter = signal.sosfilt(sos, data_pca[PCA_LABELS], axis=0)

We then try to better this result by constructing a filter pipeline, consisting of a wavelet filter, so as to keep time related information, followed by the previous Butterworth filter. In this case we adopt Daubechies 6 wavelets (db6), as they are smooth and their shape is similar to the shape of heartbeat and respiration pattern buried in SCG. Unfortunately, their maximum decomposition level is smaller than that of sym5 wavelets, so we cannot filter as precisely as above; we thus decompose up to level 8 and keep the respective approximation coefficients to reconstruct the time signal, therefore keeping frequencies in the [0, 0.39] Hz range. This method has the downside of keeping very low frequencies, but this problem is remedied by the following Butterworth filter.

In [ ]:
#Filter signal using wavelets
lvl = 8
if file_name != "center_sternum.txt":
    lvl = 7
    
coeffs = pywt.wavedec(data_pca[PCA_LABELS], "db6", level=lvl, axis=0)

#set coefficients related to undesired frequencies to zero 
for i in range(1, lvl + 1):
    coeffs[i] = np.zeros_like(coeffs[i])

#reconstruct time signal 
filtered_wt = pywt.waverec(coeffs, "db6", axis=0)

#Apply Butterworth filter
filtered_wtb = signal.sosfilt(sos, filtered_wt, axis=0)

To check whether all filters act as expected, let us plot as an example the power spectrum of the second principal component. In all three cases, the peaks around 0.2 Hz are kept while low and high frequencies are eliminated, as expected.

In [ ]:
sig_fft_wtb = fft.fftshift(fft.fft(filtered_wtb[:-1, :], axis=0))
sig_fft_butter = fft.fftshift(fft.fft(filtered_butter, axis=0))

plt.figure(figsize=(18, 5))
ax = plt.subplot(1, 1, 1)
plt.plot(sample_freq, np.abs(sig_fft[:, 1]), label=PCA_LABELS[1])
plt.plot(sample_freq, np.abs(sig_fft_wt[:, 1]),
             label="Wavelet Filtered " + PCA_LABELS[1])
plt.plot(sample_freq, np.abs(sig_fft_butter[:, 1]),
             label="Wavelet Filtered " + PCA_LABELS[1])
plt.plot(sample_freq, np.abs(sig_fft_wtb[:, 1]),
             label="Wavelet Filtered " + PCA_LABELS[1])

ax.set_xlim(0, 1.5)
plt.xticks(np.arange(0, 1, 0.1))
        
plt.xlabel("Freq [Hz]")
plt.ylabel("Power")
plt.legend(loc="best")

Now let us plot the reconstructed time signals using the filters just discussed and the one employed in the main analysis, in order to compare the reults; we also plot the original signal (rescaled by factor 2 for purely graphical reasons). 

We observe that the results for the simple Butterworth filter and those of the filter pipeline are almost identical, but both follow less closely the original signal compared to the wavelet sym5 filter. As the analysis is focused on the time related information, we conclude in the end that the latter filter is the best.

In [ ]:
#plot recostructed time signal
fig, axs = plt.subplots(nrows=N_PCA, ncols=1, figsize=(18, N_PCA*5))

for i in range(N_PCA):
    axs[i].plot(data_pca["Abs Time"], data_pca[PCA_LABELS[i]]/2, label=("Original "+PCA_LABELS[i]))
    axs[i].plot(data_pca["Abs Time"], filtered_sig[i,:-1], label=("sym5 "+PCA_LABELS[i]))
    axs[i].plot(data_pca["Abs Time"], filtered_butter[:, i], label=("Butterworth "+PCA_LABELS[i]))
    axs[i].plot(data_pca["Abs Time"], filtered_wtb[:-1, i], label=("db6 + Butterworth "+PCA_LABELS[i]))
        
    axs[i].set_xlabel("Time [s]")
    axs[i].set_ylabel("Magnitude") # ???
    axs[i].legend(loc="best")